### 0050成分股獲利數據抓取

In [2]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import datetime as dt

### 維基百科 > 0050成分股清單抓取

In [3]:
url = 'https://zh.wikipedia.org/zh-tw/%E8%87%BA%E7%81%A350%E6%8C%87%E6%95%B8'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://fubon-ebrokerdj.fbs.com.tw/',
    'Accept-Language': 'en-US,en;q=0.9',
}
res_j = re.get(url, headers=headers)
soup = BeautifulSoup(res_j.text, 'html.parser')

rows = soup.find_all('table', {'class':'wikitable'})[0].find_all('tr')

sybq = []
for i in range(1, len(rows)):
  sybq.append(rows[i].find_all('a')[1].find_all('b')[0].text)
  sybq.append(rows[i].find_all('a')[4].find_all('b')[0].text)

print(sybq)

['1101', '2883', '1216', '2884', '1301', '2885', '1303', '2886', '1326', '2887', '2002', '2890', '2207', '2891', '2301', '2892', '2303', '2912', '2308', '3008', '2317', '3017', '2327', '3034', '2330', '3037', '2345', '3045', '2357', '3231', '2379', '3661', '2382', '3711', '2395', '4904', '2412', '4938', '2454', '5871', '2603', '5876', '2609', '5880', '2880', '6446', '2881', '6505', '2882', '6669']


### 富邦證券 > 0050經營績效資料抓取

In [4]:
dfc = pd.DataFrame()

for i in sybq:

  # 用beautifulsoup 抓取網頁原始碼
  url = 'https://fubon-ebrokerdj.fbs.com.tw/z/zc/zce/zce_' + str(i) + '.djhtm'
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
      'Referer': 'https://fubon-ebrokerdj.fbs.com.tw/',
      'Accept-Language': 'en-US,en;q=0.9',
  }
  res_j = re.get(url, headers=headers)
  soup = BeautifulSoup(res_j.text, 'html.parser')


  # 抓取資料所在的tag
  rows = soup.find_all('tr')
  sybn = rows[3].find_all('td', {'class':'t10'})[0].text.split(')')[0] + ')'

  lst_t = []
  for r in rows[5:6]:
    lst = []
    for d in r.find_all('td'):
      lst.append(d.text)
    lst_t.append(lst)

  lst_t = lst_t[0]

  lst_c = []
  for r in rows[6:-2]:
    lst = []
    for d in r.find_all('td'):
      lst.append(d.text)
    lst_c.append(lst)

  lst_all = []
  for i in lst_c:
    dd={}
    for j in range(len(i)):
      dd[lst_t[j]]=i[j]
    lst_all.append(dd)

  df = pd.DataFrame(lst_all)
  df.insert(0, '股票名稱', sybn)

  # 彙整資料成為dataframe格式
  dfc = pd.concat([dfc, df], axis=0)
dfc.reset_index(inplace=True, drop=True) 

### 資料整理(文字轉數值)

In [5]:
dff = dfc.copy()
dff[dff.columns[2:]] = dff[dff.columns[2:]].astype(str).replace({',':'', '%':'', '':np.nan}, regex=True).astype(float)
dff['毛利率'] = dff['毛利率']*0.01
dff['營益率'] = dff['營益率']*0.01

### 資料整理(季別轉日期)

In [6]:
def chageDate(value):
    if isinstance(value, str):
        if '1Q' in value:
            yrs = int(value.split('.')[0]) + 1911 
            mon = 5
            dat = 15
            return dt.date(yrs, mon, dat)

        if '2Q' in value:
            yrs = int(value.split('.')[0]) + 1911 
            mon = 8
            dat = 14
            return dt.date(yrs, mon, dat)

        if '3Q' in value:
            yrs = int(value.split('.')[0]) + 1911 
            mon = 11
            dat = 14
            return dt.date(yrs, mon, dat)

        if '4Q' in value:
            yrs = int(value.split('.')[0]) + 1911 + 1
            mon = 3
            dat = 31
            return dt.date(yrs, mon, dat)

dff['季別'] = dff['季別'].map(chageDate)

### 資料匯出

In [7]:
print(dff.head(20).to_string())
dff.to_csv(os.path.join(os.getcwd() , '0050成分股_獲利能力.csv'), encoding='utf-8-sig')

        股票名稱          季別     營業收入     營業成本     營業毛利     毛利率    營業利益     營益率    業外收支     稅前淨利    稅後淨利  EPS(元)
0   台泥(1101)  2024-11-14  41075.0  31106.0   9969.0  0.2427  6270.0  0.1526   237.0   6507.0  3141.0    0.42
1   台泥(1101)  2024-08-14  38969.0  31513.0   7456.0  0.1913  4325.0  0.1110  1138.0   5463.0  2267.0    0.25
2   台泥(1101)  2024-05-15  25545.0  21657.0   3887.0  0.1522  1200.0  0.0470  2151.0   3351.0  1956.0    0.26
3   台泥(1101)  2024-03-31  28348.0  22043.0   6305.0  0.2224  3434.0  0.1211   384.0   3817.0  1802.0    0.25
4   台泥(1101)  2023-11-14  27002.0  21227.0   5775.0  0.2139  3062.0  0.1134   215.0   3277.0  1247.0    0.17
5   台泥(1101)  2023-08-14  27668.0  21683.0   5985.0  0.2163  3368.0  0.1217  2414.0   5782.0  3542.0    0.45
6   台泥(1101)  2023-05-15  26296.0  23827.0   2469.0  0.0939   166.0  0.0063  1315.0   1481.0  1407.0    0.20
7   台泥(1101)  2023-03-31  34655.0  29322.0   5333.0  0.1539  2335.0  0.0674  1471.0   3805.0  2487.0    0.36
8   台泥(1101)  2022-